In [10]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [11]:
#get data
train_data = pd.read_csv('./train.csv')
test_data = pd.read_csv('./test.csv')

y = train_data.label
X = train_data.drop(['label'],axis=1)

from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=1)

X_train = X_train/255
X_val = X_val/255
test_data = test_data/255

X_train = X_train.to_numpy().reshape(31500,28,28,1)

X_val = X_val.to_numpy().reshape(10500,28,28,1)

X_test = test_data.to_numpy().reshape(28000,28,28,1)

In [12]:
#model
import tensorflow as tf

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>0.99):
            self.model.stop_training = True
            
callbacks = myCallback()

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(28,28,1)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train.to_numpy(), epochs= 10, validation_data=(X_val,y_val.to_numpy()), callbacks=[callbacks])

Epoch 1/10
985/985 [==============================] - 10s 11ms/step - loss: 0.2160 - accuracy: 0.9348 - val_loss: 0.0905 - val_accuracy: 0.9719
Epoch 2/10
985/985 [==============================] - 9s 9ms/step - loss: 0.0632 - accuracy: 0.9795 - val_loss: 0.0567 - val_accuracy: 0.9828
Epoch 3/10
985/985 [==============================] - 9s 9ms/step - loss: 0.0459 - accuracy: 0.9855 - val_loss: 0.0559 - val_accuracy: 0.9833
Epoch 4/10
985/985 [==============================] - 9s 9ms/step - loss: 0.0335 - accuracy: 0.9894 - val_loss: 0.0448 - val_accuracy: 0.9864
Epoch 5/10
985/985 [==============================] - 10s 10ms/step - loss: 0.0252 - accuracy: 0.9920 - val_loss: 0.0436 - val_accuracy: 0.9879


In [13]:
#predictions to a csv file
preds = model.predict(X_test)
out = np.argmax(preds, axis=1)

output = pd.DataFrame({'ImageId':test_data.index+1,'Label':out})

output.to_csv('submission.csv', index=False)